In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import json
from bson.binary import Binary
from pymongo import MongoClient
from time import sleep
import pandas as pd

In [2]:
df_csv = pd.read_csv('Books.csv')
df_csv = df_csv.head()

In [3]:
class BookFinder:
    def __init__(self,isbn,NewSellerList,UsedSellerList):
        self.isbn = isbn
        self.NewSellerList = NewSellerList
        self.UsedSellerList = UsedSellerLis

In [4]:
class SellerList:
    def __init__(self,isbn,seller,middleman,country,price,notes,newflag):
        self.isbn = isbn
        self.seller = seller
        self.middleman = middleman
        self.country = country
        self.price = price
        self.notes = notes
        self.newflag = newflag

In [5]:
def getISBN(SoupObject):
    isbn = SoupObject.find("span",{"itemprop":"isbn"}).text
    print("found isbn ",isbn)
    return isbn

In [6]:
#method to get the individual line items object
def getBookDetails(SoupObject,isbn,newflag):
    #print(SoupObject.text)
    ListSellerClasses = list()
    tr = SoupObject.find_all("tr",{"class":re.compile('results-table*')})
    for item in tr:
        #print(item.text)
        td = item.find_all("td")
        price = ""
        description = ""
        middleman = ""
        country = ""
        originalSeller = ""
        #print only note first
        j = 0        
        for tdI in td:        
            #print(notes)        
            if j == 3:
                print("PRICE ->",tdI.find("a").text)
                price = tdI.find("a").text
            if j == 2:
                print("Notes - >", tdI.text)
                description = tdI.text
            if j == 1:
                if 'via' in tdI.text:
                    strSplit = tdI.text.split('via')
                    print("Middle man " ,strSplit[0])
                    print("Country", strSplit[1])
                    middleman = strSplit[0]
                    country = strSplit[1]
                else:
                    print("country only", tdI.text)
                imgTag = tdI.find('img')
                print("Original Seller ",imgTag.get('title'))
                originalSeller = imgTag.get('title')                
            j = j+1
        ListSellerClasses.append(SellerList(isbn,originalSeller,middleman,country,price,description,newflag))        
    print("process completed for isbn ",isbn)
    return ListSellerClasses

In [7]:
for index, row in df_csv.iterrows():
    isbn10 = row["Isbn10"]
    isbn13 = row["Isbn13"]
    print('isbn10 ->',isbn10,' isbn13 ->',isbn13)
    #need to see if the isbn10 has value if not try in isbn13
    url = 'https://www.bookfinder.com/search/?author=&title=&lang=en&isbn={0}&new_used=*&destination=sg&currency=SGD&mode=basic&st=sr&ac=qr'
    sleep(10)
    data = requests.get(url.format(isbn10))
    soup = BeautifulSoup(data.content, 'html.parser')
    print("Started processing with isbn10")
    isbn = getISBN(soup)
    ListUsedList = list()
    ListNewList = list()
    #print(soup)
    if not(isbn is None):
        table_result_both = soup.find_all("table",{"class":"results-table-Logo"})
        #we know that 1st table is New and Second is holding Used books
        table_result_new = table_result_both[0]
        #print(table_result_new.text)
        ListNewList = getBookDetails(table_result_new,isbn,'New') 
        ListUsedList = getBookDetails(table_result_both[1],isbn,'Used') 
    else:
        url = 'https://www.bookfinder.com/search/?author=&title=&lang=en&isbn={0}&new_used=*&destination=sg&currency=SGD&mode=basic&st=sr&ac=qr'
        data = requests.get(url.format(isbn13))
        soup = BeautifulSoup(data.content, 'html.parser')
        print("Started processing with isbn10")
        isbn = getISBN(soup)
        if not(isbn is None):
            #able to find with isbn13
            table_result_both = soup.find_all("table",{"class":"results-table-Logo"})
            table_result_new = table_result_both[0]
            ListNewList = getBookDetails(table_result_new,isbn,'New') 
            ListUsedList = getBookDetails(table_result_both[1],isbn,'Used') 
        else:
            print("No isbn found")
    #now lets try to save the data to Mongo DB
    if not ListNewList:
        print('Nothing to save for isbn10 ->',isbn10,' isbn13 ->',isbn13)
    else:
        client = MongoClient()
        db = client.BooksDB
        collection = db.col_books
        for item in ListNewList:
            collection.insert({'NewBook': item.__dict__})
            #print("Collection inserted")
        for item in ListUsedList:
            collection.insert({'UsedBook': item.__dict__})
            #print("Collection inserted")
        

isbn10 -> 0241283914  isbn13 -> 9780241283912
Started processing with isbn10
found isbn  0241283914
Middle man  The Book Depository
Country  United Kingdom
Original Seller  AbeBooks
Notes - > Hardcover, ISBN 9780241283912Publisher: Penguin Books Ltd, United Kingdom, 2016Language: English . Brand New Book. The Sunday Times bestseller The New York Times bestseller The Danish word hygge is one of those beautiful words that doesn t directly translate into English, but it more or less means comfort, warmth or togetherness. Hygge is the feeling you get when you are cuddled up on a sofa with a loved one, in warm knitted socks, in front of the fire, when it is dark, cold and stormy outside. It that feeling when you are sharing good, comfort food with your closest...
PRICE -> S$16.67
country only United Kingdom
Original Seller  Book Depository
Notes - > Hardcover, ISBN 9780241283912Publisher: Penguin Books Ltd, 2016Available - dispatched from the UK in 3 business days
PRICE -> S$17.42
Middle ma

C:\Program Files\Anaconda3\lib\site-packages\ipykernel_launcher.py:44: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
C:\Program Files\Anaconda3\lib\site-packages\ipykernel_launcher.py:47: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.


Started processing with isbn10
found isbn  1408708981
country only United Kingdom
Original Seller  Book Depository
Notes - > Hardcover, ISBN 9781408708989Publisher: Little, Brown Book Group, 2016Available - dispatched from the UK in 2 business days
PRICE -> S$16.19
Middle man  The Book Depository
Country  United Kingdom
Original Seller  AbeBooks
Notes - > Hardcover, ISBN 9781408708989Publisher: Little, Brown Book Group, United Kingdom, 2016Language: English . Brand New Book. J.K. Rowling s screenwriting debut is captured in this exciting hardcover edition of the Fantastic Beasts and Where to Find Them screenplay. When Magizoologist Newt Scamander arrives in New York, he intends his stay to be just a brief stopover. However, when his magical case is misplaced and some of Newt s fantastic beasts escape, it spells trouble for everyone.Fantastic Beasts and Where to Find Them marks the screenwriting debut of J.K. Rowling, author of...
PRICE -> S$20.56
Middle man  Book Depository Internation

Started processing with isbn10
found isbn  0752265628
Middle man  j.pop3
Country  United Kingdom
Original Seller  eBay
Notes - > ISBN 9780752265629Publisher: Pan Macmillan Australia, 2015Anti-stress book:The Mindfulness Colouring Book: Anti-stress art therapy for..... Place Of Publication: London.. Weight: 191G.
PRICE -> S$4.37
Middle man  bookwala
Country  United Kingdom
Original Seller  eBay
Notes - > ISBN 9780752265629Publisher: Pan Macmillan Australia, 2015The Mindfulness Colouring Book:Anti-stress art therapy By Emma Farrarons. Subject 1: Subjects Within Art.. Sku: Amn-Chiru-9780752265629.. Place Of Publication: London.. Weight: 191G.. Working with your hands is one of the best ways to soothe anxiety and eliminate stress. This stunning, pocket-sized colouring book offers a practical exercise in mindfulness that draws on your creativity and hones your focus. Beautifully illustrated, The Mindfulness Colouring Book is filled with templates for...
PRICE -> S$6.95
Middle man  pcc9391
C

Started processing with isbn10
found isbn  0141014083
country only United Kingdom
Original Seller  Book Depository
Notes - > Softcover, ISBN 9780141014081Publisher: Penguin Books Ltd, 2003Available - dispatched from the UK in 2 business days
PRICE -> S$23.43
Middle man  The Book Depository
Country  United Kingdom
Original Seller  AbeBooks
Notes - > Softcover, ISBN 9780141014081Publisher: Penguin Books Ltd, United Kingdom, 2003Language: English . Brand New Book. The Complete Maus by Art Spiegelman - the Pulitzer prize-winning Holocaust survivor story The most affecting and successful narrative ever done about the Holocaust Wall Street Journal The first masterpiece in comic book history The New YorkerThe Pulitzer Prize-winning Maus tells the story of Vladek Spiegelman, a Jewish survivor of Hitler s Europe, and his son, a cartoonist coming to terms with his father s story. Maus approaches the unspeakable through the...
PRICE -> S$25.27
Middle man  Blackwell's
Country  United Kingdom
Origi

Started processing with isbn10
found isbn  1408890763
Middle man  Wordery
Country  United Kingdom
Original Seller  eBay
Notes - > Hardcover, ISBN 9781408890769Publisher: Bloomsbury Children's Books, 2017Harry Potter - A History of Magic The Book of the Exhibition 9781408890769. Harry Potter - A History of Magic The Book of the Exhibition by British Library 9781408890769 (Hardback, 2017)DeliveryUK delivery is within 3 to 5 working days.International delivery varies by country, please see the Wordery store help page for details.Product detailsFormat:HardbackLanguage of text:EnglishIsbn-13:9781408890- 769, 978-1408890769Author:British LibraryPublisher:Bloomsbury Publishing PLCImp...
PRICE -> S$31.34
country only United Kingdom
Original Seller  Blackwell
Notes - > Hardcover, ISBN 9781408890769Publisher: Bloomsbury USA Academic, 2017
PRICE -> S$33.17
country only United Kingdom
Original Seller  Book Depository
Notes - > Hardcover, ISBN 9781408890769Publisher: Bloomsbury Publishing PLC, 2017